## 0. Importar librerías

In [1]:
from langchain.document_loaders import WikipediaLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import SKLearnVectorStore

## 1. Carga de documentos

In [2]:
loader = WikipediaLoader(query='Lenguaje Python',lang="es")
documents = loader.load()

/home/iabd/anaconda3/envs/ia_gpu312/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/iabd/anaconda3/envs/ia_gpu312/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [3]:
documents

[Document(metadata={'title': 'Python', 'summary': 'Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código. Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medida, programación funcional. Es un lenguaje interpretado, dinámico y multiplataforma.\nAdministrado por Python Software Foundation, posee una licencia de código abierto, denominada Python Software Foundation License.[3]\u200b Python se clasifica constantemente como uno de los lenguajes de programación más populares.[4]\u200b\n\n', 'source': 'https://es.wikipedia.org/wiki/Python'}, page_content="Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código. Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medi

In [4]:
len(documents)

24

## 2. Split de Documentos

In [5]:
# División en fragmentos
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 636, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 591, which is longer than the specified 500
Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 653, which is longer than the specified 500
Created a chunk of size 738, which is longer than the specified 500


In [6]:
len(docs)

57

## 3. Conectar a OpenAI para los embeddings

In [7]:
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()
funcion_embedding = OpenAIEmbeddings(openai_api_key=api_key)

## 4. Incrustar documentos en BD Vectores

In [8]:
persist_path="./ejemplo_wiki_bd"  #ruta donde se guardará la BBDD vectorizada

#Creamos la BBDD de vectores a partir de los documentos y la función embeddings
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=funcion_embedding,
    persist_path=persist_path,
    serializer="parquet", #el serializador o formato de la BD lo definimos como parquet
)

In [9]:
# Fuerza a guardar los nuevos embeddings en el disco
vector_store.persist()

## 5a. Consulta normal similitud coseno

In [10]:
#Creamos un nuevo documento que será nuestra "consulta" para buscar el de mayor similitud en nuestra Base de Datos de Vectores y devolverlo
consulta = "¿Por qué el lenguaje Python se llama así?"
docs = vector_store.similarity_search(consulta)
print(docs[0].page_content)

Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código. Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medida, programación funcional. Es un lenguaje interpretado, dinámico y multiplataforma.
Administrado por Python Software Foundation, posee una licencia de código abierto, denominada Python Software Foundation License.[3]​ Python se clasifica constantemente como uno de los lenguajes de programación más populares.[4]​


== Historia ==

Python fue creado a finales de los años ochenta por Guido van Rossum en Stichting Mathematisch Centrum (CWI),[5]​ en Países Bajos, como un sucesor del lenguaje de programación ABC, capaz de manejar excepciones e interactuar con el sistema operativo Amoeba.[6]​
El nombre del lenguaje proviene de la afición de su creador por los humoristas británicos Monty Python.[7]​
Guido van Rossum es el

## 5b. Consulta con compresión contextual usando LLMs

In [11]:
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [12]:
llm = ChatOpenAI(temperature=0,openai_api_key=api_key) #el parámetro temperatura define la aleatoriedad de las respuestas, temperatura = 0 significa el mínimo de aleatoriedad
compressor = LLMChainExtractor.from_llm(llm)

In [13]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vector_store.as_retriever())

In [14]:
compressed_docs = compression_retriever.invoke("¿Por qué el lenguaje Python se llama así?")

In [15]:
compressed_docs[0].page_content

'El nombre del lenguaje proviene de la afición de su creador por los humoristas británicos Monty Python.'